In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Plot parameters
plt.style.use('ggplot')
plt.rcParams['axes.facecolor']='w'

%pylab inline
pylab.rcParams['figure.figsize'] = (3, 3)

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Avoid inaccurate floating values (for inverse matrices in dot product for instance)
# See https://stackoverflow.com/questions/24537791/numpy-matrix-inversion-rounding-errors
np.set_printoptions(suppress=True)

# 2.9 The Moore-Penrose Pseudoinverse

As we have seen in [2.3]() and [2.4]() the inverse of a matrix $\boldsymbol{A}$ can be used to solve the equation $\boldsymbol{Ax}=\boldsymbol{b}$. But in the cases where the set of equation have 0 or many solutions the inverse cannot be found and the equation cannot be solved.

When there is no solution we can use the pseudoinverse to find the solution that minimize the mean square error.

# References

https://www.quora.com/What-is-the-intuition-behind-pseudo-inverse-of-a-matrix